In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import pickle
from sklearn.ensemble import RandomForestClassifier

C:\Users\lenovo\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lenovo\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\lenovo\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('train_music.csv')

In [3]:
df

,lyric,class
0,Can't drink without thinkin' about you,1
1,Now Lil Pump flyin' private jet (Yuh),0
2,"No, matter fact, you ain't help me when I had ...",0
3,"And you could find me, I ain't hidin'",0
4,From the way you talk to the way you move,1
...,...,...
51049,"I told her pour me some more, then she went ri...",0
51050,Hit the ground and crawl to the dresser,0
51051,Just keep breathin' and breathin' and breathin...,1
51052,"Down go the system, long live the king (King)",0


In [4]:
lem = WordNetLemmatizer()

df['new_lyrics'] = df['lyric'].apply(lambda x: ' '.join(lem.lemmatize(word) for word in x.split()))

In [5]:
df

,lyric,class,new_lyrics
0,Can't drink without thinkin' about you,1,Can't drink without thinkin' about you
1,Now Lil Pump flyin' private jet (Yuh),0,Now Lil Pump flyin' private jet (Yuh)
2,"No, matter fact, you ain't help me when I had ...",0,"No, matter fact, you ain't help me when I had ..."
3,"And you could find me, I ain't hidin'",0,"And you could find me, I ain't hidin'"
4,From the way you talk to the way you move,1,From the way you talk to the way you move
...,...,...,...
51049,"I told her pour me some more, then she went ri...",0,"I told her pour me some more, then she went ri..."
51050,Hit the ground and crawl to the dresser,0,Hit the ground and crawl to the dresser
51051,Just keep breathin' and breathin' and breathin...,1,Just keep breathin' and breathin' and breathin...
51052,"Down go the system, long live the king (King)",0,"Down go the system, long live the king (King)"


In [6]:
from numpy import arange
#train = MultiClassifier(random_state=42, verbose=True, cores=-1)
#model = LogisticRegression(verbose=6, n_jobs=-1, C=0.5, solver='sag')
model = RandomForestClassifier(verbose=6, max_depth=25, n_jobs=-1, n_estimators=40)


#model = train.tune_parameters(model=lr,
#                              parameters=params,
#                              tune='half-random',
#                              use_cpu=-1,
#                              verbose=4
#                              )

In [7]:
print(model)

RandomForestClassifier(max_depth=25, n_estimators=40, n_jobs=-1, verbose=6)


In [8]:



X_train, X_test, y_train, y_test = train_test_split(df['new_lyrics'], df['class'], test_size=0.1, random_state=42, stratify=df['class'])

In [9]:
count = CountVectorizer(ngram_range=(2, 3), analyzer='char', strip_accents='ascii', max_features=70000)
#count = CountVectorizer(ngram_range=(2,3))

In [10]:
count.fit(X_train)

CountVectorizer(analyzer='char', max_features=70000, ngram_range=(2, 3),
                strip_accents='ascii')

In [11]:
new = count.transform(X_train)

In [12]:
model.fit(new.toarray(), y_train)
#model.fit(new, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 40building tree 2 of 40building tree 3 of 40
building tree 4 of 40


building tree 5 of 40
building tree 6 of 40
building tree 7 of 40
building tree 8 of 40


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   12.0s


building tree 9 of 40
building tree 10 of 40
building tree 11 of 40
building tree 12 of 40
building tree 13 of 40
building tree 14 of 40
building tree 15 of 40
building tree 16 of 40
building tree 17 of 40
building tree 18 of 40
building tree 19 of 40
building tree 20 of 40
building tree 21 of 40
building tree 22 of 40
building tree 23 of 40
building tree 24 of 40
building tree 25 of 40
building tree 26 of 40
building tree 27 of 40
building tree 28 of 40
building tree 29 of 40
building tree 30 of 40
building tree 31 of 40
building tree 32 of 40
building tree 33 of 40
building tree 34 of 40
building tree 35 of 40
building tree 36 of 40
building tree 37 of 40
building tree 38 of 40
building tree 39 of 40
building tree 40 of 40


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.0min finished


RandomForestClassifier(max_depth=25, n_estimators=40, n_jobs=-1, verbose=6)

In [13]:
test = count.transform(X_test)
pred = model.predict(test.toarray())
#pred = model.predict(test)

the_train = count.transform(X_train)
pred_train = model.predict(the_train.toarray())
#pred_train = model.predict(the_train)


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    1.1s finished


In [14]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, f1_score, r2_score

print("Accuracy: ", accuracy_score(y_test, pred))
print("ROC AUC:", roc_auc_score(y_test, pred))
print("f1:", f1_score(y_test, pred))
print("r2:", r2_score(y_test, pred))

print('TRAIN SCORES')
print("Accuracy: ", accuracy_score(y_train, pred_train))
print("ROC AUC:", roc_auc_score(y_train, pred_train))
print("f1:", f1_score(y_train, pred_train))
print("r2:", r2_score(y_train, pred_train))

Accuracy:  0.836271053662358
ROC AUC: 0.8300276678231227
f1: 0.8058522991175104
r2: 0.3335403619065551
TRAIN SCORES
Accuracy:  0.9073735527117611
ROC AUC: 0.9053639220357508
f1: 0.8929954241464273
r2: 0.6229705563239449


In [15]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      2889
           1       0.83      0.78      0.81      2217

    accuracy                           0.84      5106
   macro avg       0.84      0.83      0.83      5106
weighted avg       0.84      0.84      0.84      5106



In [16]:
print(confusion_matrix(y_test, pred))

[[2535  354]
 [ 482 1735]]


In [17]:
import lzma

with lzma.open('vector_random.xz', 'wb') as f:
    pickle.dump(count, f)
    
with lzma.open('random.xz', 'wb') as g:
    pickle.dump(model, g)